In [32]:
from pl_bolts.models.self_supervised import SimCLR, SSLFineTuner
import pytorch_lightning as pl
from pl_bolts.models.regression import LogisticRegression
from pl_bolts.datamodules import STL10DataModule
import torch.nn as nn
from torchvision.datasets import STL10
from torchvision import transforms


In [12]:
weight_path = './trained_models/rgb/epoch=49-step=4800.ckpt'
simclr = SimCLR.load_from_checkpoint(weight_path, strict=False)
resnet50 = simclr.encoder
resnet50.eval()

d:\cs198\SIMCLR\simclr\lib\site-packages\pytorch_lightning\core\saving.py:225: UnderReviewWarning: The feature SimCLR is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  obj = cls(**_cls_kwargs)
d:\cs198\SIMCLR\simclr\lib\site-packages\pl_bolts\models\self_supervised\simclr\simclr_module.py:138: UnderReviewWarning: The feature resnet50 is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  return backbone(first_conv=self.first_conv, maxpool1=self.maxpool1, return_all_feature_maps=False)
d:\cs198\SIMCLR\simclr\lib\site-packages\pl_b

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [28]:
STL10_DATA_PATH = './data/'
stl10 = STL10DataModule(data_dir=STL10_DATA_PATH)
input_dim = 3 * 96 * 96

C:\Users\rayno\AppData\Local\Temp\ipykernel_13028\1326397984.py:2: UnderReviewWarning: The feature STL10DataModule is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  stl10 = STL10DataModule(data_dir=STL10_DATA_PATH)


In [29]:
import os
print(os.listdir(STL10_DATA_PATH))

['class_names.txt', 'fold_indices.txt', 'test_X.bin', 'test_y.bin', 'train_X.bin', 'train_y.bin', 'unlabeled_X.bin']


In [15]:
classifier = LogisticRegression(
    input_dim=input_dim,
    num_classes=stl10.num_classes,
    learning_rate=1e-7
)

C:\Users\rayno\AppData\Local\Temp\ipykernel_13028\3595088951.py:1: UnderReviewWarning: The feature LogisticRegression is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  classifier = LogisticRegression(


In [ ]:
class UnFreezeCallBack(pl.Callback):
    def on_epoch_end(self, trainer, pl_module):
        if trainer.current_epoch == 10.:
            encoder.unfreeze()

In [33]:
img_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

In [35]:
train_split = STL10(root=STL10_DATA_PATH, split="train", download=True, transform=img_transforms)

  0%|          | 0/2640397119 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [20]:
epochs = 5
for epoch in range(5):
    for (x, y) in stl10.val_dataloader_labeled:
        feats = resnet50(x)
        y_hat = classifier(feats)
        loss = nn.CrossEntropyLoss(y_hat, y)

    if epoch == 10:
        resnet50.unfreeze()

TypeError: 'method' object is not iterable

In [ ]:
finetuner = SSLFineTuner(simclr, in_features=simclr.z)

In [9]:
trainer = pl.Trainer(callbacks=[UnFreezeCallBack()])

trainer.fit(
    classifier,
    stl10.train_dataloader(batch_size=256),
    stl10.val_dataloader(batch_size=256)
)

GPU available: False, used: False


PicklingError: Can't pickle <function _is_device_tpu at 0x000001C9FBA589D0>: it's not the same object as lightning_lite.accelerators.tpu._is_device_tpu